<a href="https://colab.research.google.com/github/exoticTraderSisyphus/International_Parity_Conditions/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd, numpy as np
import requests
import json
import datetime as dt